# PRACTICA 2: Clasificador de tweets

En esta práctica desarrollaremos un script que se encargará de extraer información de la red social Twitter. En concreto, obtendrá los tweets de dos usuarios indicados y los almacenará en disco para, posteriormente, trabajar con ellos realizando experimentos de clasificación y análisis de sentimientos.

En primer lugar, importaremos los módulos de Python que nos harán falta para poder realizar el script. Los módulos son los siguientes:

+ twitter: Twitter API wrapper para Python.
+ ElementTree: Módulo que nos permitirá crear y leer XML.
+ CountVectorizer y TfidfVectorizer: Módulos de scikit-learn que utilizaremos para extraer características numéricas de un corpus.
+ model_selection: Módulo de scikit-learn que utilizaremos para obtener los conjuntos de entrenamiento y test.
+ SVC: Módulo de scikit-learn que utilizaremos para realizar la clasificación.
+ confusion_matrix: Módulo de scikit-learn que utilizaremos para mostrar la matriz de confusión.
+ twokenize: Módulo tokenizador de tweets.
+ pandas: Módulo de análisis de datos (sólo lo utilizaremos para obtener una representación más vistosa de la matriz de confusión).

In [1]:
# Importamos el wrapper del API de twitter
import twitter
# Importamos el parser XML
import xml.etree.ElementTree as ET
# Importamos sys y getopt para poder obtener los parametros de entrada del script
import sys, getopt
# Importamos el CountVectorizer y el TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
# Importamos model_selection para realizar el split entre muestras de entrenamiento y de test
from sklearn import model_selection
# Importamos SVC para realizar la clasificacion
from sklearn.svm import SVC
# Importamos confusion_matrix para obtener la matriz de confusion
from sklearn.metrics import confusion_matrix
# Importamos twokenize para usarlo como tokenizador de tweets
import twokenize
# Importamos pandas para la representacion en linea de comandos de la matriz de confusion como una tabla
import pandas as pd

A continuación definiremos los valores necesarios para realizar la conexión al API de Twitter. Además, crearemos variables con los valores usuarios de los que obtendremos los tweets, el número máximo de tweets que se pueden recuperar en una única petición, el tipo de vectorizador a usar, el tokenizador y el nombre del fichero donde se guardarán los tweets extraídos.

In [2]:
# Definicion de constantes como las credenciales, el maximo de tweets por peticion, valores por defecto...
CONSUMER_KEY = 'wo8oNmaGUcpF9WGckt62FtHUc'
CONSUMER_SECRET = 'do4NPAweukb41Y8qQZkpGT5IRUiKEAv6MrzYv8tl8u3NDcSK2P'
ACCESS_TOKEN = '2214334652-k6o6ODLsraogVDrcAF39hMo020aBLvwzNUF8DLl'
ACCESS_TOKEN_SECRET = 'odpZZyOlsB0TowICtmEC5LjINAIDSzHjVQolUKi6NAsZi'

MAX_TWEETS_PER_REQUEST = 200

VECTORIZER = 'tf-idf'
TOKENIZER = 'twokenize'

USER_1 = 'Pontifex'
USER_2 = 'DalaiLama'
N = 1200

# Nombre de fichero donde se guardaran los datos extraidos de twitter
FILENAME = 'pyTweetClassifier_data.xml'

También encapsularemos en funciones alguna de las funcionalidades que necesitaremos. Las funciones son las siguientes:

+ get_user_time_line: Función que descarga n tweets más recientes del usuario indicado.
+ get_XML_from_tweet: Función que, dado un tweet, crea un elemento ElementTree que formará parte de un XML.
+ get_XML_content_as_list_from_file: Función que lee un XML de tweets de un fichero y devuelve un diccionario cuya clave es el nombre de usuario y el valor una lista de sus tweets.
+ get_vectorizer: Función que devuelve un vectorizador del tipo que se le indique ('tf-idf' para TfidfVectorizer o 'count' para CountVectorizer, en otro caso devuelve None).
+ get_train_and_test_samples: Función que, a partir de un corpus de tweets, lo divide en dos partes. La primera parte, con un 80% de los tweets, será el conjunto de entrenamiento. La segunda, con el 20% restante, será el de test. Además devolverá los resultados para los dos conjuntos (a que usuario pertenece cada tweet).
+ get_positives_and_negatives: Función que toma un corpus de tweets y un lexicon y devuelve el número total de palabras, el número de palabras positivas y el de negativas (cada uno de estos valores será un diccionario en el que la clave será el nombre de usuario de los tweets y el valor el número de palabras correspondiente).

In [3]:
# Crear metodo de descarga de tweets que tenga en cuenta las limitaciones de la API
def get_user_time_line(api, user, n = MAX_TWEETS_PER_REQUEST):
    result = None
    if(n <= MAX_TWEETS_PER_REQUEST):
        result = api.GetUserTimeline(screen_name=user, count=n)
    else:
        result = []
        requests = n//MAX_TWEETS_PER_REQUEST if (n%MAX_TWEETS_PER_REQUEST == 0) else (n//MAX_TWEETS_PER_REQUEST) + 1
        max_id = None
        for count in range(1, requests+1):
            if(n < count*MAX_TWEETS_PER_REQUEST):
                res = api.GetUserTimeline(screen_name=user, count=n%MAX_TWEETS_PER_REQUEST, max_id=max_id)
                max_id = res[-1].id
                result = result + res
            else:
                res = api.GetUserTimeline(screen_name=user, count=MAX_TWEETS_PER_REQUEST, max_id=max_id)
                max_id = res[-1].id
                result = result + res
    return result

En esta función es necesario tener en cuenta el número de tweets que se desea obtener. Si este número es mayor de MAX_TWEETS_PER_REQUEST (200), se deberán hacer múltiples peticiones para obtener todos los datos especificando el id del último tweet recuperado como max_id en la nueva petición. De esta forma, la nueva petición sólo recuperará los tweets anteriores al tweet cuyo id se ha especificado.

En cuanto a las limitaciones del API sobre el número de peticiones que se pueden realizar en cierta ventana temporal, el wrapper Python-Twitter permite indicar una opción en su inicialización (se podrá ver más adelante en este notebook) para que sea él mismo el encargado de espaciarlas debidamente y cumplir con las restricciones.

In [4]:
# Funcion que parsea un tweet a XML y devuelve un elemento ElementTree
def get_XML_from_tweet(tweet):
    e = ET.Element('tweet')
    text = ET.SubElement(e, 'text')
    text.text = tweet.text
    user = ET.SubElement(e, 'user')
    user.text = tweet.user.screen_name
    return e

In [5]:
# Funcion que lee un XML de tweets de un fichero y devuelve un diccionario cuya clave es el nombre de usuario y el valor una lista de sus tweets
def get_XML_content_as_list_from_file(filename):
    parser = ET.XMLParser(encoding = 'utf-8')
    root = ET.parse(filename, parser = parser).getroot()
    result = {}
    for entry in root:
        if entry.find('text').text != None and entry.find('user').text != None:
            result[entry.find('user').text] = result[entry.find('user').text] if entry.find('user').text in result else []
            result[entry.find('user').text].append(entry.find('text').text)
    return result

In [6]:
# Funcion que devuelve un vectorizador del tipo que se le indique
def get_vectorizer(key='tf-idf', tokenizer=None):
    tok = None
    if(tokenizer != None and tokenizer == 'twokenize'):
        tok = twokenize.tokenizeRawTweetText
    if(key == 'tf-idf'):
        return TfidfVectorizer(stop_words='english', tokenizer=tok, min_df=5)
    elif(key == 'count'):
        return CountVectorizer(stop_words='english', tokenizer=tok, min_df=5)
    else:
        return None

Podemos observar en esta función que se ha sobrescrito el tokenizador a la hora de crear el vectorizador. Como esta vectorización se va a hacer sobre tweets, se ha decidido utilizar el módulo twokenizer para realizar la tokenización ya que está especialmente diseñado para este caso.

In [7]:
# Funcion que, a partir de un corpus de tweets, devuelve muestras de entrenamiento y test ademas de sus resultados (a que clase pertenece cada muestra)
def get_train_and_test_samples(corpus):
    user_class = 0
    tweets = []
    results = []
    for user, l in corpus.items():
        for tweet in l:
            tweets.append(tweet)
            results.append(user_class)
        user_class = user_class + 1

    return model_selection.train_test_split(tweets, results, test_size=0.2)

In [8]:
# Funcion que, a partir de un corpus de tweets y un lexicon, devuelve el numero total de terminos, el numero de positivos y el de negativos
def get_positives_and_negatives(corpus, lexicon):
    num_words = {}
    num_positive_words = {}
    num_negative_words = {}
    for user, l in corpus.items():
        for tweet in l:
            for token in twokenize.tokenizeRawTweetText(tweet):
                num_words[user] = num_words[user] if user in num_words else 0
                num_words[user] = num_words[user] + 1
                if token in lexicon['positive']:
                    num_positive_words[user] = num_positive_words[user] if user in num_positive_words else 0
                    num_positive_words[user] = num_positive_words[user] + 1
                elif token in lexicon['negative']:
                    num_negative_words[user] = num_negative_words[user] if user in num_negative_words else 0
                    num_negative_words[user] = num_negative_words[user] + 1
    return num_words, num_positive_words, num_negative_words

En esta función también se usa el módulo twokenize para tokenizar cada uno de los tweets.

Una vez definidas las funciones, es el momento de ejecutar el código que extraerá los datos de Twitter de los dos usuarios especificados anteriormente. El primer paso será realizar la conexión al API de Twitter:

In [9]:
api = twitter.Api(consumer_key=CONSUMER_KEY,
                      consumer_secret=CONSUMER_SECRET,
                      access_token_key=ACCESS_TOKEN,
                      access_token_secret=ACCESS_TOKEN_SECRET,
                      sleep_on_rate_limit=True)

Como se ha comentado anteriormente, es en este paso en el que es necesario especificar que sea el wrapper el encargado de espaciar las peticiones. Esto se logra indicando el parámetro sleep_on_rate_limit con valor _True_.

Creamos el elemento root del XML. El XML resultante tendrá un aspecto como este:

<pre>
&lt;?xml version="1.0" encoding="UTF-8"?&gt;
&lt;tweetList&gt;
  &lt;tweet&gt;
    &lt;text&gt;When we encounter others, do we bring them the warmth of charity or do we stay closed up and warm only ourselves before our fireplace?&lt;/text&gt;
    &lt;user&gt;Pontifex&lt;/user&gt;
  &lt;/tweet&gt;
  &lt;tweet&gt;
    &lt;text&gt;May Mary's pure and simple smile be a source of joy for each one of us as we face life&amp;#8217;s difficulties.&lt;/text&gt;
    &lt;user&gt;Pontifex&lt;/user&gt;
  &lt;/tweet&gt;
  [...]
    &lt;tweet&gt;
    &lt;text&gt;Webcast: His Holiness the Dalai Lama's Tibetan New Year Message http://bit.ly/bFcZMc&lt;/text&gt;
    &lt;user&gt;DalaiLama&lt;/user&gt;
  &lt;/tweet&gt;
  &lt;tweet&gt;
    &lt;text&gt;His Holiness the Dalai Lama in Los Angeles - 21 February 2010 http://bit.ly/cRGXyr&lt;/text&gt;
    &lt;user&gt;DalaiLama&lt;/user&gt;
  &lt;/tweet&gt;
&lt;/tweetList&gt;
</pre>

In [10]:
e = ET.Element('tweetList')

Para cada usuario, recuperamos una lista de tweets de su timeline e iteramos sobre ella creando en cada iteración un elemento &lt;tweet&gt; que contendrá el nombre de usuario y su tweet. Estos elementos se añaden a medida que se crean como hijos de &lt;tweetList&gt;.

In [11]:
for tweet in get_user_time_line(api, USER_1, N):
    e.append(get_XML_from_tweet(tweet))
for tweet in get_user_time_line(api, USER_2, N):
    e.append(get_XML_from_tweet(tweet))

Tras recuperar el contenido y haber formado el XML, lo escribimos en un fichero local.

In [12]:
f = open(FILENAME, 'w')
print(ET.tostring(e).decode('utf-8'), file=f)
f.close()

Con la información debidamente normalizada y almacenada, procederemos a trabajar con ella.
<br/>
Empezaremos recuperando la información del XML y creando un diccionario con los tweets de cada usuario.

In [13]:
# Recuperamos el contenido del fichero
corpus = get_XML_content_as_list_from_file(FILENAME)

El siguiente paso será obtener los conjuntos de entrenamiento y test (con una proporción 80%-20%) así como sus resultados.

In [14]:
# Obtenemos los conjuntos de entrenamiento y test
xTrain, xTest, yTrain, yTest = get_train_and_test_samples(corpus)

A continuación obtenemos el vectorizador.

In [15]:
# Obtenemos el vectorizador a partir de los parametros indicados
vectorizer = get_vectorizer(VECTORIZER, TOKENIZER)

Entrenamos el vectorizador con el conjunto de entrenamiento y obtenemos el vector de tfidf.

In [16]:
# Entrenamos el vectorizador con el conjunto de entrenamiento y obtenemos el vector de tfidf
xTrainVect = vectorizer.fit_transform(xTrain)

Creamos un primer clasificador SVC indicando unos parámetros aleatorios y observamos su comportamiento. Los parámetros elegidos para esta primera aproximación son C = 1.0, kernel = 'rbf', gamma = 1.0.

In [17]:
# Creamos el clasificador a partir de los parametros indicados
clf = SVC(C=1.0, kernel='rbf', gamma=1.0)

Entrenamos el clasificador con el conjunto de entrenamiento vectorizado.

In [18]:
# Entrenamos el clasificador con el conjunto de entrenamiento vectorizado
clf.fit(xTrainVect, yTrain)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=1.0, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

Obtenemos el vector de tfidf para el conjunto de test y utilizamos el clasificador para predecir los resultados para dicho conjunto.

In [19]:
# Obtenemos el vector de tfidf para el conjunto de test
xTestVect = vectorizer.transform(xTest)
# Predecimos los resultados del conjunto de test
yTestPred = clf.predict(xTestVect)

Con los resultados obtenidos, obtenemos la matriz de confusión y el valor de accuracy.

In [20]:
matrix = confusion_matrix(yTest, yTestPred)
pd.DataFrame(matrix, ['Real Negativo', 'Real Positivo'], ['Pred. Negativo', 'Pred. Positivo'])

,Pred. Negativo,Pred. Positivo
Real Negativo,213,14
Real Positivo,9,244


In [21]:
print('Accuracy: {:.2f}%'.format(clf.score(xTestVect, yTest)*100))

Accuracy: 95.21%


Aunque se puede observar que el porcentaje de acierto con este caso es bueno. Es interesante jugar con los parámetros disponibles para crear el clasificador y así poder comparar sus resultados. Por lo tanto, se mostrarán algunas variantes del código anterior.

C = 3.5, kernel = 'rbf', gamma=1.0

In [22]:
# Creamos el clasificador a partir de los parametros indicados
clf = SVC(C=3.5, kernel='rbf', gamma=1.0)
# Entrenamos el clasificador con el conjunto de entrenamiento vectorizado
clf.fit(xTrainVect, yTrain)
# Predecimos los resultados del conjunto de test
yTestPred = clf.predict(xTestVect)
matrix = confusion_matrix(yTest, yTestPred)
pd.DataFrame(matrix, ['Real Negativo', 'Real Positivo'], ['Pred. Negativo', 'Pred. Positivo'])

,Pred. Negativo,Pred. Positivo
Real Negativo,214,13
Real Positivo,8,245


In [23]:
print('Accuracy: {:.2f}%'.format(clf.score(xTestVect, yTest)*100))

Accuracy: 95.62%


C = 7.0, kernel = 'rbf', gamma=1.0

In [24]:
# Creamos el clasificador a partir de los parametros indicados
clf = SVC(C=7.0, kernel='rbf', gamma=1.0)
# Entrenamos el clasificador con el conjunto de entrenamiento vectorizado
clf.fit(xTrainVect, yTrain)
# Predecimos los resultados del conjunto de test
yTestPred = clf.predict(xTestVect)
matrix = confusion_matrix(yTest, yTestPred)
pd.DataFrame(matrix, ['Real Negativo', 'Real Positivo'], ['Pred. Negativo', 'Pred. Positivo'])

,Pred. Negativo,Pred. Positivo
Real Negativo,214,13
Real Positivo,8,245


In [25]:
print('Accuracy: {:.2f}%'.format(clf.score(xTestVect, yTest)*100))

Accuracy: 95.62%


C = 1.0, kernel = 'linear', gamma=1.0

In [26]:
# Creamos el clasificador a partir de los parametros indicados
clf = SVC(C=1.0, kernel='linear', gamma=1.0)
# Entrenamos el clasificador con el conjunto de entrenamiento vectorizado
clf.fit(xTrainVect, yTrain)
# Predecimos los resultados del conjunto de test
yTestPred = clf.predict(xTestVect)
matrix = confusion_matrix(yTest, yTestPred)
pd.DataFrame(matrix, ['Real Negativo', 'Real Positivo'], ['Pred. Negativo', 'Pred. Positivo'])

,Pred. Negativo,Pred. Positivo
Real Negativo,214,13
Real Positivo,9,244


In [27]:
print('Accuracy: {:.2f}%'.format(clf.score(xTestVect, yTest)*100))

Accuracy: 95.42%


C = 3.5, kernel = 'linear', gamma=1.0

In [28]:
# Creamos el clasificador a partir de los parametros indicados
clf = SVC(C=3.5, kernel='linear', gamma=1.0)
# Entrenamos el clasificador con el conjunto de entrenamiento vectorizado
clf.fit(xTrainVect, yTrain)
# Predecimos los resultados del conjunto de test
yTestPred = clf.predict(xTestVect)
matrix = confusion_matrix(yTest, yTestPred)
pd.DataFrame(matrix, ['Real Negativo', 'Real Positivo'], ['Pred. Negativo', 'Pred. Positivo'])

,Pred. Negativo,Pred. Positivo
Real Negativo,218,9
Real Positivo,9,244


In [29]:
print('Accuracy: {:.2f}%'.format(clf.score(xTestVect, yTest)*100))

Accuracy: 96.25%


C = 7.0, kernel = 'linear', gamma=1.0

In [30]:
# Creamos el clasificador a partir de los parametros indicados
clf = SVC(C=7.0, kernel='linear', gamma=1.0)
# Entrenamos el clasificador con el conjunto de entrenamiento vectorizado
clf.fit(xTrainVect, yTrain)
# Predecimos los resultados del conjunto de test
yTestPred = clf.predict(xTestVect)
matrix = confusion_matrix(yTest, yTestPred)
pd.DataFrame(matrix, ['Real Negativo', 'Real Positivo'], ['Pred. Negativo', 'Pred. Positivo'])

,Pred. Negativo,Pred. Positivo
Real Negativo,213,14
Real Positivo,13,240


In [31]:
print('Accuracy: {:.2f}%'.format(clf.score(xTestVect, yTest)*100))

Accuracy: 94.38%


C = 3.5, kernel = 'poly', degree=2, gamma=1.0

In [32]:
# Creamos el clasificador a partir de los parametros indicados
clf = SVC(C=3.5, kernel='poly', degree=2, gamma=1.0)
# Entrenamos el clasificador con el conjunto de entrenamiento vectorizado
clf.fit(xTrainVect, yTrain)
# Predecimos los resultados del conjunto de test
yTestPred = clf.predict(xTestVect)
matrix = confusion_matrix(yTest, yTestPred)
pd.DataFrame(matrix, ['Real Negativo', 'Real Positivo'], ['Pred. Negativo', 'Pred. Positivo'])

,Pred. Negativo,Pred. Positivo
Real Negativo,212,15
Real Positivo,9,244


In [33]:
print('Accuracy: {:.2f}%'.format(clf.score(xTestVect, yTest)*100))

Accuracy: 95.00%


Podemos observar que utilizando valores de C y kernels diferentes se obtienen valores de accuracy parecidos pero no iguales. Para intentar buscar la mejor configuración para el clasificador podemos realizar lo siguiente:

In [34]:
# Importamos el modulo GridSearchCV
from sklearn.grid_search import GridSearchCV
# Importamos numpy
import numpy as np
# Indicamos los hiperparametros que queremos evaluar
hyperParams = {'C': np.arange(1.0, 3.5, 0.5), 
               'kernel': ['linear', 'poly', 'rbf'],
               'degree': np.arange(1, 4, 1),
               'gamma': np.arange(0.5, 1.5, 0.2)}
# Creamos el clasificador
modelCV = GridSearchCV(SVC(), hyperParams, cv=4, scoring='accuracy')
# Lo entrenamos
modelCV.fit(xTrainVect, yTrain)
# Obtenemos los mejores hiperparametros
print("Mejores hiperparametros: ", modelCV.best_params_)

/home/edgar/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/edgar/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


Mejores hiperparametros:  {'C': 1.0, 'degree': 1, 'gamma': 0.89999999999999991, 'kernel': 'rbf'}


Utilizamos los parámetros obtenidos para realizar la clasificación como en los casos anteriores.

In [35]:
# Creamos el clasificador a partir de los parametros indicados
clf = SVC(C=modelCV.best_params_['C'], kernel=modelCV.best_params_['kernel'], gamma=modelCV.best_params_['gamma'], degree=modelCV.best_params_['degree'])
# Entrenamos el clasificador con el conjunto de entrenamiento vectorizado
clf.fit(xTrainVect, yTrain)
# Predecimos los resultados del conjunto de test
yTestPred = clf.predict(xTestVect)
matrix = confusion_matrix(yTest, yTestPred)
pd.DataFrame(matrix, ['Real Negativo', 'Real Positivo'], ['Pred. Negativo', 'Pred. Positivo'])

,Pred. Negativo,Pred. Positivo
Real Negativo,213,14
Real Positivo,9,244


In [36]:
print('Accuracy: {:.2f}%'.format(clf.score(xTestVect, yTest)*100))

Accuracy: 95.21%


Como se puede observar, GridSearch no tiene por qué devolver unos parámetros que proporcionen el mayor valor de accuracy al predecir los resultados del conjunto de test. Sin embargo, como se hace uso de validación cruzada, si que podemos confiar en que los parámetros son fiables y pueden dar buenos valores para diferentes conjuntos de test.

Tras realizar un análisis de los datos obtenidos podemos llegar a la conclusión de que, a través del clasificador SVC, se puede realizar una clasificación de los tweets de dos usuarios obteniendo un nivel bastante alto de fiabilidad. En este caso en concreto, se han utilizado los tweets de dos líderes religiosos, que son el Papa Francisco y el Dalai Lama. Como la temática de sus tweets suele ser parecida, se podría llegar a pensar que sería un inconveniente para el clasificador. Sin embargo, su valor de accuracy se sitúa por encima del 95% en la mayoría de los casos. Por lo tanto, este ejemplo parece una buena métrica para medir el buen hacer del clasificador y podemos pensar que, con usuarios de perfil más variado, sus resultados serían todavía mejores.

Por último se hará un pequeño experimento de análisis de sentimiento básico en el que, utilizando un lexicon de términos positivos y negativos, se medirá la positividad/negatividad de los tweets de cada usuario.

En primer lugar se recupera el lexicon con los términos positivos y negativos.

In [37]:
# Recuperamos el lexicon de terminos positivos y negativos
pos_neg_words = {'positive': [], 'negative': []}

for pos_word in open('positive-words.txt', 'r').readlines()[35:]:
    pos_neg_words['positive'].append(pos_word.rstrip())

for neg_word in open('negative-words.txt', 'r').readlines()[35:]:
    pos_neg_words['negative'].append(neg_word.rstrip())

A continuación obtenemos para cada usuario el número total de palabras de su conjunto de tweets, el número de palabras positivas y el número de negativas.

In [38]:
num_words, positives, negatives = get_positives_and_negatives(corpus, pos_neg_words)

Por último, con los datos obtenidos en el paso anterior, podemos obtener los porcentajes de términos positivos y negativos para cada usuario.

In [39]:
print('Porcentaje de terminos positivos para el usuario {}: {:.2f}%'.format(USER_1, positives[USER_1]*100/num_words[USER_1]))
print('Porcentaje de terminos negativos para el usuario {}: {:.2f}%'.format(USER_1, negatives[USER_1]*100/num_words[USER_1]))
print()
print('Porcentaje de terminos positivos para el usuario {}: {:.2f}%'.format(USER_2, positives[USER_2]*100/num_words[USER_2]))
print('Porcentaje de terminos negativos para el usuario {}: {:.2f}%'.format(USER_2, negatives[USER_2]*100/num_words[USER_2]))

Porcentaje de terminos positivos para el usuario Pontifex: 5.55%
Porcentaje de terminos negativos para el usuario Pontifex: 2.51%

Porcentaje de terminos positivos para el usuario DalaiLama: 6.95%
Porcentaje de terminos negativos para el usuario DalaiLama: 2.03%


Se puede observar que en ambos casos el porcentaje de términos positivos es mayor que el de términos negativos. Esto nos hace pensar que la mayoría de los tweets de estos usuarios transmiten mensajes positivos, lo que concuerda con su perfil (líder religioso). Como mejora para obtener porcentajes mayores en el análisis de sentimiento se podrían filtrar stopwords, ya que tienen una ocurrencia alta y provocan que el cómputo total de palabras sea muy elevado. De todas formas, sin este filtrado y teniendo en cuenta lo anterior, los resultados obtenidos son suficientemente significativos.